In [1]:
from games_setup import *
import SBMLLint.common.constants as cn
from SBMLLint.common.reaction import Reaction
from SBMLLint.common.stoichiometry_matrix import StoichiometryMatrix
from SBMLLint.games.som import SOM
from SBMLLint.games.mesgraph import MESGraph
from SBMLLint.games.games_pp import GAMES_PP, SOMStoichiometry, SOMReaction, TOLERANCE
from SBMLLint.games.games_report import GAMESReport, SimplifiedReaction
import collections
import tesbml
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import time

Current Directory: /Users/woosubs/Desktop/ModelEngineering/SBMLLint/SBMLLint/notebook


In [2]:
data_dir=cn.BIOMODELS_DIR
#files = [f for f in os.listdir(data_dir) if f[-4:] == ".xml"]
files = [f for f in os.listdir(data_dir) if f[:7] == "BIOMD00"]
paths = [os.path.join(data_dir, filename) for filename in files]

In [3]:
data_dir

'/Users/woosubs/Desktop/ModelEngineering/SBMLLint/SBMLLint/data/biomodels'

In [4]:
len(files)

656

In [5]:
# statistics columns
NUM_REACTIONS = "num_reactions(nonbdry)"
LP_ERROR = "lp_error"
GAMES_ERROR = "games_error"
GAMESPP_ERROR = "gamespp_error"
TYPEI_ERROR = "type1_error"
TYPEII_ERROR = "type2_error"
CANCELING_ERROR = "canceling_error"
ECHELON_ERROR = "echelon_error"
TYPEIII_ERROR = "type3_error"
result_columns = [NUM_REACTIONS,
                  LP_ERROR,
                  GAMES_ERROR,
                  GAMESPP_ERROR,
                  TYPEI_ERROR,
                  TYPEII_ERROR,
                  CANCELING_ERROR,
                  ECHELON_ERROR,
                  TYPEIII_ERROR]
## invertible matrix column? 
# INVERTIBLE = "l_inverse"

In [6]:
results = pd.DataFrame(0, index=files, columns=result_columns)
results[:5]

,num_reactions(nonbdry),lp_error,games_error,gamespp_error,type1_error,type2_error,canceling_error,echelon_error,type3_error
BIOMD0000000199_url.xml,0,0,0,0,0,0,0,0,0
BIOMD0000000189_url.xml,0,0,0,0,0,0,0,0,0
BIOMD0000000387_url.xml,0,0,0,0,0,0,0,0,0
BIOMD0000000397_url.xml,0,0,0,0,0,0,0,0,0
BIOMD0000000413_url.xml,0,0,0,0,0,0,0,0,0


In [7]:
simple = SimpleSBML()
simple.initialize(os.path.join(data_dir, "BIOMD0000000244_url.xml"))
s = StoichiometryMatrix(simple)
consistent = s.isConsistent()
print("consistent? ", consistent)

consistent?  False


//anaconda3/envs/sbmllint_env/lib/python3.6/site-packages/scipy/optimize/_linprog_util.py:763: OptimizeWarning:

A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.



In [8]:
# LP only
simple = SimpleSBML()
count = 0
lp_start = time.time()
for file in files:
  count += 1
  if (count%100)==0:
    print("we are analyzing Model number:", count)
  try:
    simple.initialize(os.path.join(data_dir, file))
    s = StoichiometryMatrix(simple)
    num_reactions = s.stoichiometry_matrix.shape[1]
    results.at[file, NUM_REACTIONS] = num_reactions
    if num_reactions:
      consistent = s.isConsistent()
    else:
      consistent = -1
    results.at[file, LP_ERROR] = 1 - int(consistent)
  except:
    results.at[file, LP_ERROR] = -1
lp_end = time.time()
lp_time = lp_end - lp_start
print("Analysis finished!")
print("LP time:", lp_time)

/Users/woosubs/Desktop/ModelEngineering/SBMLLint/SBMLLint/SBMLLint/common/stoichiometry_matrix.py:88: OptimizeWarning:

Solving system with option 'cholesky':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'cholesky' to False.

/Users/woosubs/Desktop/ModelEngineering/SBMLLint/SBMLLint/SBMLLint/common/stoichiometry_matrix.py:88: OptimizeWarning:

Solving system with option 'sym_pos':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'sym_pos' to False.

//anaconda3/envs/sbmllint_env/lib/python3.6/site-packages/scipy/optimize/_linprog_ip.py:110: LinAlgWarning:

Ill-conditioned matrix (rcond=1.99514e-18): result may not be accurate.

//anaconda3/envs/sbmllint_env/lib/python3.6/site-packages/scipy/optimize/_linprog_ip.py:110: LinAlgWarning:

Ill-conditioned matrix (rco

we are analyzing Model number: 100


//anaconda3/envs/sbmllint_env/lib/python3.6/site-packages/scipy/optimize/_linprog_ip.py:110: LinAlgWarning:

Ill-conditioned matrix (rcond=1.86384e-35): result may not be accurate.

//anaconda3/envs/sbmllint_env/lib/python3.6/site-packages/scipy/optimize/_linprog_ip.py:110: LinAlgWarning:

Ill-conditioned matrix (rcond=6.83193e-21): result may not be accurate.

//anaconda3/envs/sbmllint_env/lib/python3.6/site-packages/scipy/optimize/_linprog_ip.py:110: LinAlgWarning:

Ill-conditioned matrix (rcond=4.11351e-35): result may not be accurate.

//anaconda3/envs/sbmllint_env/lib/python3.6/site-packages/scipy/optimize/_linprog_ip.py:110: LinAlgWarning:

Ill-conditioned matrix (rcond=6.72697e-21): result may not be accurate.

//anaconda3/envs/sbmllint_env/lib/python3.6/site-packages/scipy/optimize/_linprog_ip.py:110: LinAlgWarning:

Ill-conditioned matrix (rcond=5.83777e-20): result may not be accurate.

//anaconda3/envs/sbmllint_env/lib/python3.6/site-packages/scipy/optimize/_linprog_ip.py:11

we are analyzing Model number: 200


//anaconda3/envs/sbmllint_env/lib/python3.6/site-packages/scipy/optimize/_linprog_ip.py:110: LinAlgWarning:

Ill-conditioned matrix (rcond=1.28673e-17): result may not be accurate.



...

we are analyzing Model number: 500


//anaconda3/envs/sbmllint_env/lib/python3.6/site-packages/scipy/optimize/_linprog_ip.py:110: LinAlgWarning:

Ill-conditioned matrix (rcond=3.72045e-18): result may not be accurate.

//anaconda3/envs/sbmllint_env/lib/python3.6/site-packages/scipy/optimize/_linprog_ip.py:110: LinAlgWarning:

Ill-conditioned matrix (rcond=3.67095e-36): result may not be accurate.

//anaconda3/envs/sbmllint_env/lib/python3.6/site-packages/scipy/optimize/_linprog_ip.py:110: LinAlgWarning:

Ill-conditioned matrix (rcond=6.20275e-36): result may not be accurate.

//anaconda3/envs/sbmllint_env/lib/python3.6/site-packages/scipy/optimize/_linprog_ip.py:110: LinAlgWarning:

Ill-conditioned matrix (rcond=1.10128e-35): result may not be accurate.

//anaconda3/envs/sbmllint_env/lib/python3.6/site-packages/scipy/optimize/_linprog_ip.py:110: LinAlgWarning:

Ill-conditioned matrix (rcond=3.58365e-19): result may not be accurate.

//anaconda3/envs/sbmllint_env/lib/python3.6/site-packages/scipy/optimize/_linprog_ip.py:11

we are analyzing Model number: 600


//anaconda3/envs/sbmllint_env/lib/python3.6/site-packages/scipy/optimize/_linprog_ip.py:110: LinAlgWarning:

Ill-conditioned matrix (rcond=4.77536e-21): result may not be accurate.

//anaconda3/envs/sbmllint_env/lib/python3.6/site-packages/scipy/optimize/_linprog_ip.py:110: LinAlgWarning:

Ill-conditioned matrix (rcond=1.25833e-21): result may not be accurate.



Analysis finished!
LP time: 46.06716799736023


In [9]:
lp_results = results[results[LP_ERROR] == 1]
len(lp_results)
print("(Mean) ISS for LP is:", np.mean(lp_results[NUM_REACTIONS]))
print("(STD) ISS for LP is:", np.std(lp_results[NUM_REACTIONS]))

(Mean) ISS for LP is: 55.51048951048951
(STD) ISS for LP is: 94.38982308045166


In [10]:
len(results[results[LP_ERROR]==1])

143

In [11]:
len(results[results[LP_ERROR]==-1])

5

In [12]:
# GAMES only
simple = SimpleSBML()
count = 0
games_start = time.time()
for file in files:
  count += 1
  if (count%100)==0:
    print("we are analyzing Model number:", count)
  try:
    simple.initialize(os.path.join(data_dir, file))
    m = GAMES_PP(simple)
    if simple.reactions:
      res = m.analyze(simple_games=True, error_details=False)
      results.at[file, GAMES_ERROR] = int(res)
      if res:
        gr = GAMESReport(m)
        summary = m.error_summary
        if m.type_one_errors:
          results.at[file, TYPEI_ERROR] = len(m.type_one_errors)
          report, error_num = gr.reportTypeOneError(m.type_one_errors, explain_details=True)
        if m.type_two_errors:
          results.at[file, TYPEII_ERROR] = len(m.type_two_errors)
          report, error_num = gr.reportTypeTwoError(m.type_two_errors, explain_details=True)
  except:
    results.at[file, GAMES_ERROR] = -1   
games_end = time.time()
games_time = games_end - games_start
print("Analysis finished!")
print("GAMES time:", games_time)

we are analyzing Model number: 100
we are analyzing Model number: 200
we are analyzing Model number: 300
we are analyzing Model number: 400
we are analyzing Model number: 500
we are analyzing Model number: 600
Analysis finished!
GAMES time: 106.94337201118469


In [13]:
print(games_time)

106.94337201118469


In [14]:
print("number of detected errors: ", len(results[results[GAMES_ERROR]==1]))
print("number of GAMES but not in LP", len(results[(results[GAMES_ERROR]==1) & (results[LP_ERROR]!=1)]))

number of detected errors:  109
number of GAMES but not in LP 0


In [15]:
len(results[results[GAMES_ERROR]==-1])

5

In [43]:
# GAMES+
# file, GAMES_ERROR coding:
# 0; normal - no error found
# -1; not loaded or error found
# 1; normal - error found
# 2; echelon error found, but it is not explainable
# 3; type III error found, but it is not explainable
simple = SimpleSBML()
count = 0
gamespp_start = time.time()
for file in files:
  count += 1
  if (count%100)==0:
    print("we are analyzing Model number:", count)
  try:
    simple.initialize(os.path.join(data_dir, file))
    m = GAMES_PP(simple)
    if simple.reactions:
      res = m.analyze(simple_games=False, error_details=False)
      results.at[file, GAMESPP_ERROR] = int(res)
      if res:
        condition_number = np.linalg.cond(m.lower)
        if condition_number > 300:
          print("*****The L matrix of the model %s has a condition number %f*****" % (file, condition_number))
        gr = GAMESReport(m)
        summary = m.error_summary
        if m.type_one_errors:
          results.at[file, TYPEI_ERROR] = len(m.type_one_errors)
          report, error_num = gr.reportTypeOneError(m.type_one_errors, explain_details=True)
        if m.type_two_errors:
          results.at[file, TYPEII_ERROR] = len(m.type_two_errors)
          report, error_num = gr.reportTypeTwoError(m.type_two_errors, explain_details=True)
        if m.canceling_errors:
          results.at[file, CANCELING_ERROR] = len(m.canceling_errors)
          report, error_num = gr.reportCancelingError(m.canceling_errors, explain_details=True)
        if m.echelon_errors:
          results.at[file, ECHELON_ERROR] = len(m.echelon_errors)
          report, error_num = gr.reportEchelonError(m.echelon_errors, explain_details=True)
          if report is False:
            results.at[file, GAMESPP_ERROR] = 2
            print("Model %s has an inexplainable Echelon Error" % file)
            print("As the lower matrix has a condition number %f" % condition_number)
            print("Decide if the matrix is invertible")
        if m.type_three_errors:
          results.at[file, TYPEIII_ERROR] = len(m.type_three_errors)
          report, error_num = gr.reportTypeThreeError(m.type_three_errors, explain_details=True)
          if report is False:
            results.at[file, GAMESPP_ERROR] = 3
            print("Model %s has an inexplainable Type III Error" % file)
            print("As the lower matrix has a condition number %f" % condition_number)
            print("Decide if the matrix is invertible")
  except:
    results.at[file, GAMES_ERROR] = -1   
gamespp_end = time.time()
gamespp_time = gamespp_end - gamespp_start
print("\nAnalysis finished!")
print("GAMES++ time:", gamespp_time)

Model BIOMD0000000478_url.xml has an inexplainable Echelon Error
As the lower matrix has a condition number 8.590581
Decide if the matrix is invertible
Model BIOMD0000000478_url.xml has an inexplainable Type III Error
As the lower matrix has a condition number 8.590581
Decide if the matrix is invertible
we are analyzing Model number: 100
The L matrix of the model BIOMD0000000628_url.xml has a condition number 501.451879.
Model BIOMD0000000628_url.xml has an inexplainable Echelon Error
As the lower matrix has a condition number 501.451879
Decide if the matrix is invertible
Model BIOMD0000000628_url.xml has an inexplainable Type III Error
As the lower matrix has a condition number 501.451879
Decide if the matrix is invertible
we are analyzing Model number: 200
we are analyzing Model number: 300
we are analyzing Model number: 400
The L matrix of the model BIOMD0000000574_url.xml has a condition number 718.276667.
Model BIOMD0000000574_url.xml has an inexplainable Echelon Error
As the lowe

In [44]:
print("Lower matrix has a condition number %f" % np.linalg.cond(m.lower))

LinAlgError: 0-dimensional array given. Array must be at least two-dimensional

In [45]:
print("number of detected errors: ", len(results[results[GAMESPP_ERROR]==1]))
print("number of GAMES errors not in LP", len(results[(results[GAMESPP_ERROR]==1) & (results[LP_ERROR]!=1)]))
len(results[results[GAMESPP_ERROR]==-1])

number of detected errors:  133
number of GAMES errors not in LP 0


0

In [46]:
len(results[results[GAMESPP_ERROR]==2])

2

In [47]:
len(results[results[GAMESPP_ERROR]==3])

3

In [20]:
results[results[GAMESPP_ERROR]==3]

,num_reactions(nonbdry),lp_error,games_error,gamespp_error,type1_error,type2_error,canceling_error,echelon_error,type3_error
BIOMD0000000426_url.xml,29,1,0,3,0,0,0,10,4
BIOMD0000000628_url.xml,584,0,0,3,0,0,0,29,7
BIOMD0000000574_url.xml,576,0,0,3,0,0,0,15,10
BIOMD0000000243_url.xml,19,1,0,3,0,0,0,2,1


In [52]:
simple = load_file_from_games(478)
m = GAMES_PP(simple)
res = m.analyze(simple_games=False, error_details=True)

We just analyzed the data...
Type I error:  []
Type II error:  []
Canceling error:  []
Echelon error:  []
Type III error:  []


In [53]:
np.isfinite(np.linalg.cond(m.lower))

True

In [54]:
np.linalg.det(m.lower) == 1.0

True

In [34]:
gr = GAMESReport(m)
echelon_report, error_num = gr.reportEchelonError(m.echelon_errors, explain_details=True)
print(echelon_report)
print(error_num)


We detected a mass imbalance
:  -> species_32

from the following reactions.

1. reaction_24: species_31 + species_18 -> species_32 + species_19
2. reaction_14: species_6 -> species_16 + species_27
3. reaction_17: species_12 + species_3 -> species_28 + species_4
4. reaction_23: species_30 + species_3 -> species_31 + species_4
5. reaction_18: species_24 + species_23 -> species_24 + species_22
6. reaction_20: species_5 + species_4 -> species_6 + species_3
7. reaction_29: species_4 -> species_3 + species_23
8. reaction_16: species_16 + species_19 + species_23 -> species_12 + species_18

----------------------------------------------------------------------

These uni-uni reactions created mass-equivalence.
(The molecules within a curly bracket have the same atomic mass.)

9. reaction_21: species_28 -> species_29;   {species_28=species_29}
10. reaction_22: species_29 -> species_30;   {species_28=species_29=species_30}
11. reaction_15: species_16 -> species_27;   {species_16=species_27}
12

In [55]:
np.linalg.det(np.linalg.inv(m.lower))

1.0

In [37]:
# # GAMES++ 
# simple = SimpleSBML()
# count = 0
# gamespp_start = time.time()
# for file in files[82:89]:
#   print("We are doing file", file)
#   count += 1
#   if (count%100)==0:
#     print("we are analyzing Model number:", count)
#   simple.initialize(os.path.join(data_dir, file))
#   m = GAMES_PP(simple)
#   if simple.reactions:
#     res = m.analyze(simple_games=False, error_details=False)
#     results.at[file, GAMESPP_ERROR] = int(res)
#     if res:
#       gr = GAMESReport(m)
#       summary = m.error_summary
#       if m.type_one_errors:
#         results.at[file, TYPEI_ERROR] = len(m.type_one_errors)
#         report, error_num = gr.reportTypeOneError(m.type_one_errors)
#       if m.type_two_errors:
#         results.at[file, TYPEII_ERROR] = len(m.type_two_errors)
#         report, error_num = gr.reportTypeTwoError(m.type_two_errors)
#       if m.canceling_errors:
#         results.at[file, CANCELING_ERROR] = len(m.canceling_errors)
#         report, error_num = gr.reportCancelingError(m.canceling_errors)
#       if m.echelon_errors:
#         results.at[file, ECHELON_ERROR] = len(m.echelon_errors)
#         report, error_num = gr.reportEchelonError(m.echelon_errors)
#       if m.type_three_errors:
#         results.at[file, TYPEIII_ERROR] = len(m.type_three_errors)
#         report, error_num = gr.reportTypeThreeError(m.type_three_errors)

We are doing file BIOMD0000000032_url.xml
We are doing file BIOMD0000000022_url.xml
We are doing file BIOMD0000000479_url.xml
We are doing file BIOMD0000000501_url.xml
We are doing file BIOMD0000000469_url.xml
We are doing file BIOMD0000000511_url.xml
We are doing file BIOMD0000000354_url.xml


In [28]:
results.loc["BIOMD0000000479_url.xml", LP_ERROR]

1

In [23]:
results.head()

,num_reactions(nonbdry),lp_error,games_error,gamespp_error,type1_error,type2_error,canceling_error,echelon_error,type3_error
BIOMD0000000199_url.xml,10,0,0,0,0,0,0,0,0
BIOMD0000000189_url.xml,13,1,0,1,4,0,0,0,0
BIOMD0000000387_url.xml,4,0,0,0,0,0,0,0,0
BIOMD0000000397_url.xml,29,0,0,0,0,0,0,0,0
BIOMD0000000413_url.xml,5,1,0,1,1,0,0,0,0


In [35]:
import networkx as nx
som_example = list(m.nodes)[-1]
PathNodesReactions = collections.namedtuple('PathNodesReactions',
    'node1 node2 reactions')
def getSOMPath(som, mole1, mole2):
  """
  Create an undirected graph between
  two molecules within a SOM
  and find the shortest path
  :param SOM som:
  :param str mole1:
  :param str mole2:
  :return PathNodesReactions result_data:
  """   
  molecule1 = simple.getMolecule(mole1).name
  moelcule2 = simple.getMolecule(mole2).name
  # construct undirected graph
  subg = nx.Graph()
  # here, every reaction is 1-1 reaction
  for reaction in list(som.reactions):
    print(reaction.makeIdentifier(is_include_kinetics=False))
    node1 = reaction.reactants[0].molecule.name
    node2 = reaction.products[0].molecule.name
    if subg.has_edge(node1, node2):
      reaction_label = subg.get_edge_data(node1, node2)[cn.REACTION]
      # if reaction.label is not already included in the attribute,
      if reaction.label not in set(reaction_label):
        reaction_label = reaction_label + [reaction.label]
    else:
      reaction_label = [reaction.label]    
    subg.add_edge(node1, node2, reaction=reaction_label)
  print(list(subg.edges))
  path = [p for p in nx.shortest_path(subg, 
                                      source=mole1, 
                                      target=mole2)]
  # if result has more than 1 element need a for loop
  print("We found the shortest path from " + mole1 + " to " + mole2)
  result_data = []
  for idx in range(len(path)-1):
    print(path[idx] + "=" + path[idx+1], end=" ")
    edge_reactions = subg.get_edge_data(path[idx], path[idx+1])[cn.REACTION]
    print("by reaction(s)", edge_reactions)
    result_data.append(PathNodesReactions(node1=path[idx], 
                                          node2=path[idx+1],
                                          reactions=edge_reactions))
  return result_data

In [131]:
som_path = getSOMPath(som_example, 's260', 's270')
type(som_path)

r58: s36 -> s232
re65: s260 -> s232
re64: s270 -> s232
[('s36', 's232'), ('s232', 's260'), ('s232', 's270')]
We found the shortest path from s260 to s270
s260=s232 by reaction(s) ['re65']
s232=s270 by reaction(s) ['re64']


list

In [134]:
for pat in som_path:

  print(pat.node1 + " = " + pat.node2 + " by", end=" ")
  for r in pat.reactions:
    reaction = simple.getReaction(r)
    print(reaction.makeIdentifier(is_include_kinetics=False))

s260 = s232 by re65: s260 -> s232
s232 = s270 by re64: s270 -> s232


In [124]:
print(subg.edges)
print(subg.has_edge('s260', 's232'))
print(subg.has_edge('s232', 's260'))

[('s36', 's232'), ('s232', 's260'), ('s232', 's270')]
True
True


In [87]:
som = som_example
mole1 = 's260'
mole2 = 's270'
molecule1 = simple.getMolecule(mole1).name
moelcule2 = simple.getMolecule(mole2).name
# construct undirected graph
subg = nx.Graph()
# subg.add_nodes_from(som.molecules)
# here, every reaction is 1-1 reaction
for reaction in list(som.reactions):
  print(reaction.makeIdentifier(is_include_kinetics=False))
  node1 = reaction.reactants[0].molecule.name
  node2 = reaction.products[0].molecule.name
  if subg.has_edge(node1, node2):
    reaction_label = subg.get_edge_data(node1, node2)[cn.REACTION]
    # if reaction.label is not already included in the attribute,
    if reaction.label not in set(reaction_label):
      reaction_label = reaction_label + [reaction.label]
  else:
    reaction_label = [reaction.label]    
  subg.add_edge(node1, node2, reaction=reaction_label)


r58: s36 -> s232
re65: s260 -> s232
re64: s270 -> s232


In [11]:
import tesbml
for error_path in error_paths:
  document = tesbml.readSBML(error_path)
  model = document.getModel()
  #pm.print_model(model)
  try:
    simple.initialize(error_path)
  except:
    print(error_path)
    print("showed error")

/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/BIOMD0000000596_url.xml
showed error
/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/MODEL0568648427_url.xml
showed error
/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/BIOMD0000000410_url.xml
showed error
/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/BIOMD0000000081_url.xml
showed error
/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/BIOMD0000000075_url.xml
showed error
/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/BIOMD0000000094_url.xml
showed error
/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/BIOMD0000000353_url.xml
showed error
/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/BIOMD0000000627_url.xml
showed error
/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/data/MODEL0072364382_url.xml
showed error


In [7]:
#pm.print_model(model)
for rct in model.getReaction(0).getListOfReactants():
  print(rct.getSpecies())
  print("similar, ", rct.species)
  print(rct.stoichiometry)
  print("similar, ", rct.getStoichiometry())
print("rct done! now pdt")
for pdt in model.getReaction(0).getListOfProducts():
  print(pdt.getSpecies())
  print(pdt.stoichiometry)
  print("similar, ", pdt.getStoichiometry())
pm.print_model(model)

model.getReaction(3).getId()
[rct.stoichiometry for r in model.getReaction(3).getListOfReactants() \
if rct.species != "haha"]
#r = Reaction(model.getReaction(3))
#r.category

rct done! now pdt
s51
1.0
similar,  1.0
<Model MODEL1508180000 "Philipson2015 - Innate immune response modulated by NLRX1">
MyD88_a_HP:  -> s51;
MyD88_d: s51 -> ;
TRAF_d: s7 -> ;
NFkB_a1:  -> s4;
NFkB_d: s4 -> ;
CytoL_a:  -> s34;
CytoL_d: s34 -> ;
NLRX1_d: s49 -> ;
MyD88_a:  -> s51;
HP_a:  -> s18;
HP_CytoL: s18 -> ;
NLRX1_TF:  -> s49;
NFkB_a2:  -> s4;
RIG_a_HP:  -> RIG;
RIG_d: RIG -> ;
MAVS_a:  -> MAVS;
MAVS_d: MAVS -> ;
IRF_a1:  -> IRF;
IRF_a2:  -> IRF;
IRF_a3:  -> IRF;
IRF_d: IRF -> ;
IFN_a:  -> IFN;
IFN_d: IFN -> ;
NOD_a:  -> NOD1;
NOD_d: NOD1 -> ;
HP_IFN: s18 -> ;
TRAF_a:  -> s7;
TFa_a:  -> TFa;
TFa_d: TFa -> ;
TFi_a:  -> TFi;
TFi_d: TFi -> ;
CytoE_a:  -> CytoE;
CytoE_d: CytoE -> ;
X_a:  -> X;
X_d: X -> ;
HP_CytoE: s18 -> ;


[]

In [56]:
REACTION_1_1 = "reaction_1_1"
REACTION_n_1 = "reaction_n_1"
REACTION_1_n = "reaction_1_n"
REACTION_n_n = "reaction_n_n"
REACTION_BOUNDARY = "reaction_boundary"
ReactionCategory = collections.namedtuple('ReactionCategory',
    'category predicate')
REACTION_CATEGORIES = [
    ReactionCategory(category=REACTION_1_1,
        predicate=lambda x,y,z,w: (x==1) and (y==1) and (z==w)),
    ReactionCategory(category=REACTION_1_n,
        predicate=lambda x,y,z,w: ((x==1) and (y>1) and (z==1.00)) or ((x==1) and (y==1) and (z<w))),
    ReactionCategory(category=REACTION_n_1,
        predicate=lambda x,y,z,w: ((x>1) and (y==1) and (w==1.00)) or ((x==1) and (y==1) and (z>w))),
    ReactionCategory(category=REACTION_n_n,
        predicate=lambda x,y,z,w: ((x>1) and (y>1)) or ((x==1) and (y>1) and (z!=1.00)) or ((x>1) and (y==1) and (w!=1.00))),
    ReactionCategory(category=REACTION_BOUNDARY,
        predicate=lambda x,y,z,w: (x==0) or (y==0) or (z==0) or (w==0)),
    ]

In [57]:
for reaction in model.getListOfReactions():
  num_rct = len([r.species for r in reaction.getListOfReactants()])
  num_pdt = len([p.species for p in reaction.getListOfProducts()])
  stoi_rct = sum([r.stoichiometry for r in reaction.getListOfReactants()])
  stoi_pdt = sum([p.stoichiometry for p in reaction.getListOfProducts()])
  print("x =", num_rct, ";y =", num_pdt, ";z =", stoi_rct, ";w =", stoi_pdt)
  for reaction_category in REACTION_CATEGORIES:
    if reaction_category.predicate(num_rct, num_pdt, 
                                 stoi_rct, stoi_pdt):
      print(reaction_category.category)

x = 3 ;y = 1 ;z = 4.0 ;w = 2.0
reaction_n_n
x = 3 ;y = 1 ;z = 4.0 ;w = 1.5
reaction_n_n
x = 1 ;y = 1 ;z = 1.0 ;w = 1.0
reaction_1_1
x = 1 ;y = 1 ;z = 1.0 ;w = 1.0
reaction_1_1


In [49]:
pm.print_model(model)

<Model "Arnold2011_Damour2007_RuBisCO-CalvinCycle">
PGA_prod_Vc: RuBP + CO2 + NADPH -> PGA;
PGA_prod_Vo: RuBP + O2 + NADPH -> PGA;
PGA_cons: PGA -> RuBP;
NADPH_prod: NADP -> NADPH;


In [51]:
x = 3 ;y = 1 ;z = 4.0 ;w = 2.0
((x>1) and (y==1) and (w!=1.00))

True

In [1]:
def make_autopct(values):
    def my_autopct(pct):
        total = sum(values)
        val = int(round(pct*total/100.0))
        return '{p:.1f}%  ({v:d})'.format(p=pct,v=val)
    return my_autopct

labels = ['Mass Balance Issue', 
          'No Mass Balance Issue', 
          'Only Boundary Reactions', 'Not Loaded']
sizes = [146, 365, 209, 9]
explode = (0, 0.0, 0, 0)  # only "explode" the 2nd slice (i.e. 'Hogs')
colors = ['orangered', 'green', 'lightgrey', 'black']
fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode=explode, labels=labels, autopct=make_autopct(sizes),
        colors=colors, shadow=True, startangle=50)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()

NameError: name 'plt' is not defined

In [2]:
plt.pie(sizes, labels=labels, colors=colors, 
        autopct=make_autopct(sizes), startangle=70, radius=1.3)
txt = "[Analysis of Mass Balance for BioModels]"
plt.figtext(0.5, 0.01, txt, wrap=True, horizontalalignment='center', fontsize=12)

NameError: name 'plt' is not defined